In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

path = '/content/drive/MyDrive/NTU/Courses/CZ4041/'
xTrain=pd.read_csv(path+'cleaned_train.csv')
xTest=pd.read_csv(path+'cleaned_test.csv')

In [ ]:
yTrain = xTrain['price_doc'].apply(lambda x: np.log1p(x))
xTrain.drop(columns=['id','timestamp','price_doc'],axis=1,inplace=True)

print(xTrain.shape)

(30471, 279)


In [ ]:
#Split train data into 2 parts
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(xTrain, yTrain, test_size=0.15,random_state=42)

In [ ]:
print(X_tr.isnull().values.any())
print(X_val.isnull().values.any())

False
False


In [ ]:
from sklearn import preprocessing

numerical_cols = xTrain.select_dtypes(exclude=['object'])
categorical_cols = xTrain.select_dtypes(include=['object']).copy()

# Encoding for categorical columns:
for c in categorical_cols:
  le = preprocessing.LabelEncoder()
  le.fit(X_tr[c])

  X_val[c] = X_val[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  le.classes_ = np.append(le.classes_, '<unknown>')

  X_tr[c] = le.transform(X_tr[c])
  X_val[c] = le.transform(X_val[c])

# Scaling for numercial columns:
for c in numerical_cols:
  min = X_tr[c].min()
  max = X_tr[c].max()

  X_tr[c] = (X_tr[c] - min)/(max-min)
  X_val[c] = (X_val[c] - min)/(max-min)

In [ ]:
print(X_tr.isnull().values.any())
print(X_val.isnull().values.any())

False
False


In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error

In [ ]:
kcv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
lasso_cv = LassoCV(alphas=None, cv=kcv, max_iter=10000)
lasso_cv.fit(X_tr, y_tr)

LassoCV(cv=RepeatedKFold(n_repeats=5, n_splits=10, random_state=42),
        max_iter=10000)

In [ ]:
print("Best alpha:", lasso_cv.alpha_)

Best alpha: 0.23500321021261555


In [ ]:
lasso = Lasso(alpha=lasso_cv.alpha_)
lasso.fit(X_tr, y_tr)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.226e+03, tolerance: 9.481e-01
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.23500321021261555)

In [ ]:
yPredictionTrain = lasso.predict(X_tr)
yPredictionTest = lasso.predict(X_val)

In [ ]:
# Calculating RMSE
rmse_train = np.sqrt(mean_squared_error(y_tr, yPredictionTrain))
rmse_test = np.sqrt(mean_squared_error(y_val, yPredictionTest))

print("Training RMSE:", rmse_train)
print("Test RMSE:", rmse_test)

Training RMSE: 0.5960365393057545
Test RMSE: 0.8028243751421273


In [ ]:
# Coefficients for each features
coeff_df = pd.DataFrame(lasso.coef_, xTrain.columns, columns=['Coefficient'])
print(coeff_df)

                         Coefficient
full_sq                     0.000000
life_sq                     0.000000
floor                       0.000000
max_floor                   0.000000
material                    0.000000
...                              ...
n_sales_month               0.000000
subarea_dist_to_kremlin    -0.000000
count_nan_per_row          -0.000000
apt_name                   -0.000156
apt_name_yrmo               0.000075

[279 rows x 1 columns]


#Predicting prices for test.csv data

In [ ]:
testID = xTest['id']
xTest.drop(['id', 'timestamp'],axis=1,inplace=True)

print(xTest.shape)

(7662, 279)


In [ ]:
num = xTrain.select_dtypes(exclude=['object'])
cat = xTrain.select_dtypes(include=['object']).copy()


for c in cat:
  le = preprocessing.LabelEncoder()
  le.fit(xTrain[c])

  xTest[c] = xTest[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
  le.classes_ = np.append(le.classes_, '<unknown>')

  xTest[c] = le.transform(xTest[c])

for c in num:
  min = xTrain[c].min()
  max = xTrain[c].max()

  xTest[c] = (xTest[c] - min)/(max-min)

In [ ]:
#predict the price
# Testing the Model

test_pred = lasso.predict(xTest)

test_pred = np.array(test_pred)
test_pred = test_pred.T
print(test_pred.shape)

(7662,)


In [ ]:
xTest['price_doc'] = np.expm1(test_pred)
xTest.head(10)
xTest['id'] = testID
print(xTest[['id','price_doc']])

         id     price_doc
0     30474  2.319056e+07
1     30475  1.098744e+07
2     30476  1.207389e+07
3     30477  1.087292e+07
4     30478  1.103565e+07
...     ...           ...
7657  38131  2.138282e+07
7658  38132  1.173878e+07
7659  38133  7.558772e+06
7660  38134  1.318504e+07
7661  38135  3.078219e+07

[7662 rows x 2 columns]


In [ ]:
# Dumb the test data with predicted price in to the csv file
xTest[['id','price_doc']].to_csv('output_file2.csv',index=False)

# New Section